# 최적 파라미터 도출
* b=height^((1)/(-(end-head)))
* c=height^((1)/(head-start))
* g(x)=b^(x-end)
* f(x)=c^(x-start)
* 위 함수의 결과 중 교차되는 값보다 작은 값들을 사용하는 연령분포모형
* https://www.geogebra.org/m/b7sesmgy

In [67]:
import pandas as pd
import numpy as np
import time
import iddModel.learnUsingNumpy as idoctor
import multiprocessing
import parmap 
import math
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)
def runGetCost(tuningValueList):
    resultData = []
    
    for value in tuningValueList:
        resultData.append([value ,idoctor.getCost(value)])
        
        
    return resultData
def startGetCost(inputData):
    tRate1 = inputData[0]
    tRate2 = inputData[1]
    tRate3 = inputData[2]
    tRetire1 = inputData[3]
    tRetire2 = inputData[4]
    
    dataSize = len(tRate1)*len(tRate2)*len(tRate3)*len(tRetire1)*len(tRetire2)
    timeMinute = dataSize*22/1000/60/22
    curentData = 0
    resultData = []
    
    num_cores = 30

    now = time.localtime()
    print("│ Start Time : %04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec))
    print("│ expect running time : " + str(round(timeMinute)) + "min")
    print("│ Data Size = " + str(dataSize))
    print("└───────────────────────")
    print('│ '+str(round(curentData/dataSize*100,2))+'%')
    
    tuningList = []
    
    for b1 in tRate1:
        for b2 in tRate2:
            for b3 in tRate3:
                for c1 in tRetire1:
                    for c2 in tRetire2:
                        ageTunList = [[25,40,26,b1],
                                      [27,40,28,b2],
                                      [26,40,27,b3]]
                        retireTunList = [c1,35,c2]
                        tuningList.append([ageTunList,retireTunList])

    
    splitedTuningList = np.array_split(tuningList, num_cores) 
    splitedTuningList = [i.tolist() for i in splitedTuningList] 
    
    now = time.localtime()
    
    result = parmap.map(runGetCost, splitedTuningList, pm_pbar=True, pm_processes=num_cores)
    
    print("| End Time : %04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec))
    return result

# 파라미터 테스트
* 코스트 세트 반환
* tuningSetAgeRate : 연령분포모델 파라미터 세트
* tuningSetRetireRate : 은퇴율 파라미터 세트

In [68]:
tuningSetAgeRate = [[25,40,26.5,1.1],
                      [27,40,27,11.1],
                      [26,40,27,11.1]]
tuningSetRetireRate = [1.09,35,1]
%time npCostData = idoctor.getCost([tuningSetAgeRate, tuningSetRetireRate])
pd.DataFrame(npCostData)

CPU times: user 17.2 ms, sys: 3.14 ms, total: 20.3 ms
Wall time: 18.1 ms


,0,1,2,3,4,5,6,7,8,9,10,11
0,99,2958,4804,653,2667,4343,0.0281247,0.0341738,0.0492188,0.0177174,0.0387197,0.0550803
1,3132,1627,331,650,306,218,0.0281247,0.0188107,0.000133866,0.00502674,0.00238592,0.00203524
2,360,2093,3608,672,1745,3644,0.0294669,0.0232762,0.0349341,0.0162038,0.0262991,0.0451743
3,3237,1195,168,998,569,307,0.0294669,0.0133539,0.00165196,0.0092133,0.00547562,0.00307611
4,1895,885,1872,794,970,2888,0.0281119,0.00962396,0.019071,0.011137,0.0125249,0.0331449
5,2713,593,469,1310,855,424,0.0281119,0.00648723,0.00627709,0.0140236,0.00926054,0.00461761
6,2423,227,935,815,217,2099,0.030534,0.00266782,0.00912944,0.0102078,0.00335098,0.0234369
7,2916,24,482,1717,1156,549,0.030534,0.000369056,0.00588322,0.0184008,0.0124566,0.0059289
8,3003,1400,6,796,394,1195,0.0320998,0.0149748,5.77276e-05,0.00850742,0.00422048,0.0127803
9,3000,865,326,2060,1446,685,0.0320998,0.00925232,0.00348366,0.0220385,0.0154693,0.00732802


# 트레이닝
* 트레이닝값 설정 후 실행

In [66]:
tAge1 = np.arange(26, 27, 1)   # 연령분포모델 변수2 c/ 범위 0.3~1
tAge2 = np.arange(28, 32, 1)   # 연령분포모델 변수2 c/ 범위 0.3~1
tAge3 = np.arange(27, 28, 1)   # 연령분포모델 변수2 c/ 범위 0.3~1
tRate1 = np.arange(20, 110, 10)   # 연령분포모델 변수2 c/ 범위 0.3~1
tRate2 = np.arange(100, 1000, 100)   # 연령분포모델 변수2 c/ 범위 0.3~1
tRate3 = np.arange(100, 1000, 100)   # 연령분포모델 변수2 c/ 범위 0.3~1
tRetire1 = np.arange(1.1, 1.55, 0.05)  # 은퇴율모델 변수1 / 범위 1 이상
tRetire2 = np.arange(0.5, 10.5, 0.5) # 은퇴율모델 변수3 / 범위 0 이상
    
liCostData = startGetCost([tRate1, tRate2, tRate3, tRetire1, tRetire2])

│ Start Time : 2020/11/20 22:34:38
│ expect running time : 2min
│ Data Size = 131220
└───────────────────────
│ 0.0%


100%|██████████| 30/30 [04:25<00:00,  8.84s/it]  


| End Time : 2020/11/20 22:34:39


## 최적 코스트값 계산
* 위에서 계산한 결과에서 최적 코스트 도출
* minManCost : 남성 최소 코스트값
* minWomanCost : 여성 최소 코스트값
* minManTuningValue : 남성 최소 코스트 도출 파라미터 값
* minWomanTuningValue : 여성 최소 코스트 도출 파라미터 값

In [77]:
npCostData = np.array(liCostData)
np.shape(npCostData)
minManCost = (np.concatenate((npCostData[0][0][1][0],npCostData[0][0][1][2],npCostData[0][0][1][4],npCostData[0][0][1][6],npCostData[0][0][1][8],npCostData[0][0][1][10]))).reshape(6,12).sum(axis=0)
minWomanCost = (np.concatenate((npCostData[0][0][1][1],npCostData[0][0][1][3],npCostData[0][0][1][5],npCostData[0][0][1][7],npCostData[0][0][1][9],npCostData[0][0][1][11]))).reshape(6,12).sum(axis=0)
minManTuningValue = npCostData[0][0][0]
minWomanTuningValue = npCostData[0][0][0]
count = 0
for data1 in npCostData:
    for data2 in data1:   
        count += 1
        npManCost = (np.concatenate((data2[1][0],data2[1][2],data2[1][4],data2[1][6],data2[1][8],data2[1][10]))).reshape(6,12).sum(axis=0)
        npWomanCost = (np.concatenate((data2[1][1],data2[1][3],data2[1][5],data2[1][7],data2[1][9],data2[1][11]))).reshape(6,12).sum(axis=0)

        manCostCount = 0
        womanCostCount = 0
        
        for i in range(0,12):
            if npManCost[i]<=minManCost[i]:
                manCostCount += 1
            else:
                break
                
        for i in range(0,12):
            if npWomanCost[i]<=minWomanCost[i]:
                womanCostCount += 1
            else:
                break
                
        if manCostCount==12:
            minManCost = npManCost
            minManTuningValue = data2[0]
            
        if womanCostCount==12:
            minWomanCost = npWomanCost
            minWomanTuningValue = data2[0]

pd.DataFrame([[minManCost, minManTuningValue],[minWomanCost,minWomanTuningValue]])
print(count)

131220


In [78]:
minManCost = np.around(minManCost.astype(np.double),2)
minWomanCost = np.around(minWomanCost.astype(np.double),2)
pd.DataFrame([minManCost,minWomanCost])

,0,1,2,3,4,5,6,7,8,9,10,11
0,24643.0,12766.0,21541.0,2641.0,8531.0,8396.0,0.15,0.13,0.14,0.16,0.13,0.11
1,27240.0,7648.0,3721.0,6373.0,5563.0,7181.0,0.15,0.08,0.02,0.04,0.04,0.07


In [79]:
dfManParam1 = pd.DataFrame(minManTuningValue[0])
dfWomanParam1 = pd.DataFrame(minWomanTuningValue[0])
dfManParam2 = pd.DataFrame(minManTuningValue[1]).T
dfWomanParam2 = pd.DataFrame(minWomanTuningValue[1]).T
dfManParam1

,0,1,2,3
0,25,40,26,20
1,27,40,28,100
2,26,40,27,100


# 로그 응용 모형 단일 테스트
* 로그 응용 모형의 파라미터 값의 형태를 보기위함
* 파라미터별로 20대 비율이 얼마나 나오는지 테스트
* for문에 사용되는 m과 m r값의 변화에 따른 20대 비율 체크

In [24]:
idoctor.makeLogModel2([27,40,29,10])[25:30].sum()

0.47249500786521914

In [30]:
import plotly.graph_objects as go
import numpy as np
dataList = []
for m in range(27,30):
    for n in range(1,2000,10):
            dataList.append([idoctor.makeLogModel2([26, 40, m, n]), [m,n]])
            sumList = []
for data in dataList:
    sumList.append([(data[0][25:30].sum()),data[1][0], data[1][1]])
sumList2 = np.array(sumList).T
# Helix equation
t = np.linspace(0, 20, 100)
x, y, z = np.cos(t), np.sin(t), t

fig = go.Figure(data=[go.Scatter3d(
    x=sumList2[1],
    y=sumList2[2],
    z=sumList2[0],
    mode='markers',
    marker=dict(
        size=3,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()
